# Coursera Capstone Project Using Seattle Collision Data

### This is a Jupyter notebook we will be using to analyze and present findings based on collision data in the city of Seattle

# Introduction and Business Understanding

## Overview

There are more than 10,000 traffic collisions per year involving cars, bicyclists and pedestrians. Understanding the causes of collisions as well as the conditions that impact their severity will help provide insight to officials on how to better allocate resources to help reduce the number and severity of such incidents.

Further, a better understanding of the factors that increase the likelihood of collisions and increase the probability of injury or property damage can help with education efforts to help individuals take greater precautions when making travel decisions.

## Goals of the Project

The goal of the project is to use publicly available data compiled by the Seattle Deport of Transportation (SDOT), to identify feautures in the dataset that yield predictive information on the number and severity of collisions and injuries in Seattle.

We will also look to use data visualization tools to communicate this information and provide an overview of the current state of traffic collisions in Seattle.

# Data Understanding

The dataset we are using is *Collisions - All Years* dataset maintained by the SDOT Traffic Management Division's Traffic Records Group.  This dataset includes all types of collisions, including car, bicycle, and pedestrian as provided by the Seattle Police Department in their Traffic Records.

The data set contains information on over 194,000 collisions in Seattle over a 15-year period.  The primary attribute we are looking to predict is the severity of the collision as captured by the Severity Code assigned to the collision.  Interestingly, the dataset description provided by SDOT indicates this Severity Code attribute should take values between 0 and 3 (including both 2 and 2b to differential "injury" from "serious injury"); however, the actual data set only contains the values 1 and 2 for this attribute.  One avenue to explore in a future project is to find additional information on this target attribute.

The data includes 37 different features including: day, time, month, lighting conditions, road conditions and weather conditions.

A full description of the data can be found at: https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Metadata.pdf

In [ ]:
# Importing some packages we'll want.

import pandas as pd
import numpy as np
import folium
import matplotlib.pyplot as plt

import pylab as pl
import scipy.optimize as opt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

%matplotlib inline

In [ ]:
path="~/Documents/CertificationStuff/IBMPythonDataScience/Data_Science_Capstone/Data-Collisions.csv"

df = pd.read_csv(path, low_memory=False)

In [ ]:
# Drop rows without latitude and longitude
df.dropna(subset=["X"],axis=0,inplace=True)

# Replace the rows that are missing Coordinates with the average coordinate 
#mean_longtitude = df["X"].mean()
#mean_latitude = df["Y"].mean()
#df["X"].replace(np.nan, mean_longitude, inplace = True)
#df["Y"].replace(np.nan, mean_latitutde, inplace = True)

# Drop Unnecessary or Redundant Columns
df.drop(['SEVERITYCODE.1','OBJECTID','INCKEY','COLDETKEY','REPORTNO','STATUS','ADDRTYPE','INTKEY','LOCATION','EXCEPTRSNCODE','EXCEPTRSNDESC','SDOT_COLCODE','SDOT_COLDESC','PEDROWNOTGRNT','SDOTCOLNUM','ST_COLCODE','ST_COLDESC','SEGLANEKEY','CROSSWALKKEY'],axis = 1, inplace = True)

df['SEVERITYCODE'] = df['SEVERITYCODE'] - 1

# Replace string values with Boolean values in some appropriate Columns
df['INATTENTIONIND'].replace(np.nan, 0, inplace = True)
df['INATTENTIONIND'].replace(to_replace='Y', value = 1, inplace = True)

df['UNDERINFL'].replace(np.nan, 0, inplace = True)
df['UNDERINFL'].replace('N', 0, inplace = True)
df['UNDERINFL'].replace('*',0, inplace= True)
df['UNDERINFL'].replace('Y',1,inplace = True)
df['UNDERINFL'] = df['UNDERINFL'].astype(int)

df['COLLISIONTYPE'].replace(np.nan, "Other", inplace = True)

df['SPEEDING'].replace(np.nan, False, inplace = True)
df['SPEEDING'].replace('Y', True, inplace = True)
df['SPEEDING'] = df['SPEEDING'].astype(int)

df['HITPARKEDCAR'].replace('N', False, inplace = True)
df['HITPARKEDCAR'].replace('Y', True, inplace = True)
df['HITPARKEDCAR'] = df['HITPARKEDCAR'].astype(int)

# Consolidate missing, "Unknown", NaN values in some columns to "Unknown"
df['WEATHER'].replace(np.nan,"Unknown", inplace = True)
df['WEATHER'].replace("Other", "Unknown", inplace = True)

df['ROADCOND'].replace(np.nan, "Unknown", inplace = True)
df['ROADCOND'].replace("Other", "Unknown", inplace = True)

df['LIGHTCOND'].replace(np.nan, "Unknown", inplace = True)
df['LIGHTCOND'].replace("Other", "Unknown", inplace = True)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.describe(include="all")

In [ ]:
possible_severities_counts = df['SEVERITYDESC'].value_counts().to_frame()
print(possible_severities_counts)
print()

possible_weather_conditions = df['WEATHER'].value_counts().to_frame()
print(possible_weather_conditions)
print()

possible_road_conditions = df['ROADCOND'].value_counts().to_frame()
print(possible_road_conditions)
print()

possible_lighting_conditions = df['LIGHTCOND'].value_counts().to_frame()
print(possible_lighting_conditions)
print()

speeding = df['SPEEDING'].value_counts().to_frame()
print(speeding)
print()

under_influence = df['UNDERINFL'].value_counts().to_frame()
print(under_influence)
print()

collision_type = df['COLLISIONTYPE'].value_counts().to_frame()
print(collision_type)

# print("Possible Severities: ", df['SEVERITYDESC'].unique()) 
# print("Weather Conditions: ", df['WEATHER'].unique())
# print("Road Conditions: ", df['ROADCOND'].unique())
# print("Lighting Conditions: ", df['LIGHTCOND'].unique())

In [ ]:
# Put some types of accidents into data frames so we can analyze

df_icy_road = df[df['ROADCOND']=='Ice']
df_wet_road = df[df['ROADCOND']=='Wet']
df_snow_road = df[df['ROADCOND']=='Snow/Slush']

df_speeding = df[df['SPEEDING'] == 1]
df_under_influence = df[df['UNDERINFL'] == 1]

df_dark = df[df['LIGHTCOND'] == 'Dark - Street Lights Off']

In [ ]:
# latitude and longitude of center of Seattle
latitude = 47.6062
longitude = -122.3321

# Create map Seattle
seattle_map = folium.Map(location=[latitude,longitude],zoom_start=11)

# Display map of Seattle
seattle_map

In [ ]:
# Adding features to the map

dark_collision = folium.map.FeatureGroup()

for lat,lng in zip(df_dark.Y, df_dark.X):
    dark_collision.add_child(
        folium.features.CircleMarker(
            [lat,lng],
            radius = 5,
            color = 'yellow',
            fill = True,
            fill_color = 'blue',
            fill_opacity = 0.6
        )
    )

# snow_collisions = folium.map.FeatureGroup()

# for lat,lng in zip(df_snow_road.Y, df_snow_road.X):
#    snow_collisions.add_child(
#        folium.features.CircleMarker(
#            [lat,lng],
#            radius = 5,
#            color='yellow',
#            fill = True,
#            fill_color='blue',
#            fill_opacity = 0.6
#        )
#    )

seattle_map.add_child(dark_collision)

In [ ]:
from folium import plugins

seattle_map = folium.Map(location=[latitude,longitude],zoom_start=11)

dark_collision = plugins.MarkerCluster().add_to(seattle_map)

for lat, lng, label in zip (df_dark.Y, df_dark.X, df_dark.SEVERITYDESC):
    folium.Marker(
        location = [lat,lng],
        icon = None,
        popup = label,
    ).add_to(dark_collision)
    
seattle_map

In [ ]:
"""

All the code to create a visualize a decision tree for predicting accident severity.  It didn't seem to yield much results.


!conda install -c conda-forge pydotplus -y
!conda install -c conda-forge python-graphviz -y

from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn.externals.six import StringIO
import pydotplus
import matplotlib.image as mpimg
from sklearn import tree
%matplotlib inline

df_decision = df[['SEVERITYDESC','WEATHER','ROADCOND','LIGHTCOND','SPEEDING','UNDERINFL']]
df_decision = df_decision[df_decision.WEATHER != 'Unknown']
df_decision = df_decision[df_decision.ROADCOND != 'Unknown']
df_decision = df_decision[df_decision.LIGHTCOND != 'Unknown']


X = df_decision[['WEATHER','ROADCOND','LIGHTCOND','SPEEDING','UNDERINFL']].values

le_weather = preprocessing.LabelEncoder()
le_weather.fit(['Blowing Sand/Dirt', 'Clear', 'Fog/Smog/Smoke', 'Overcast', 'Partly Cloudy', 'Raining', 'Severe Crosswind', 'Sleet/Hail/Freezing Rain', 'Snowing'])
X[:,0] = le_weather.transform(X[:,0])

le_road = preprocessing.LabelEncoder()
le_road.fit(['Dry','Wet','Ice','Snow/Slush','Standing Water','Sand/Mud/Dirt','Oil'])
X[:,1] = le_road.transform(X[:,1])

le_light = preprocessing.LabelEncoder()
le_light.fit(['Daylight','Dark - Street Lights On','Dusk','Dawn','Dark - No Street Lights','Dark - Street Lights Off','Dark - Unknown Lighting'])
X[:,2] = le_light.transform(X[:,2])

X[0:5]

y = df_decision['SEVERITYDESC']
y[0:5]

X_trainset, X_testset, y_trainset, y_testset = train_test_split(X,y,test_size = 0.3, random_state = 3)

severityTree = DecisionTreeClassifier(criterion = "entropy", max_depth = 4)
severityTree

severityTree.fit(X_trainset, y_trainset)

predictionTree = severityTree.predict(X_testset)

print (predictionTree [0:5])
print (y_testset [0:5])

print ("Decision Tree Accuracy: ", metrics.accuracy_score(y_testset, predictionTree))

dot_data = StringIO()
filename = "severityTree.png"
featureNames = ['WEATHER','ROADCOND','LIGHTCOND','SPEEDING','UNDERINFL']
targetNames = df["SEVERITYDESC"].unique().tolist()
out = tree.export_graphviz(severityTree,feature_names = featureNames, out_file = dot_data, class_names = np.unique(y_trainset), filled = True, special_characters = True, rotate = False)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_png(filename)
img = mpimg.imread(filename)
plt.figure(figsize = (100,200))
plt.imshow(img, interpolation = 'nearest')
"""

In [ ]:
import pylab as pl
import scipy.optimize as opt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [ ]:
severity_df = df[['SEVERITYCODE','COLLISIONTYPE','UNDERINFL','WEATHER','ROADCOND','LIGHTCOND','SPEEDING']]

severity_df = severity_df[severity_df.WEATHER != 'Unknown']
severity_df = severity_df[severity_df.ROADCOND != 'Unknown']
severity_df = severity_df[severity_df.LIGHTCOND != 'Unknown']

X = np.asarray(severity_df[['COLLISIONTYPE','UNDERINFL','WEATHER','ROADCOND','LIGHTCOND','SPEEDING']])

le_collision_type = preprocessing.LabelEncoder()
le_collision_type.fit(['Parked Car','Angles','Rear Ended','Other','Sideswipe','Left Turn','Pedestrian','Cycles','Right Turn','Head On'])
X[:,0] = le_collision_type.transform(X[:,0])

le_weather = preprocessing.LabelEncoder()
le_weather.fit(['Blowing Sand/Dirt', 'Clear', 'Fog/Smog/Smoke', 'Overcast', 'Partly Cloudy', 'Raining', 'Severe Crosswind', 'Sleet/Hail/Freezing Rain', 'Snowing'])
X[:,2] = le_weather.transform(X[:,2])

le_road = preprocessing.LabelEncoder()
le_road.fit(['Dry','Wet','Ice','Snow/Slush','Standing Water','Sand/Mud/Dirt','Oil'])
X[:,3] = le_road.transform(X[:,3])

le_light = preprocessing.LabelEncoder()
le_light.fit(['Daylight','Dark - Street Lights On','Dusk','Dawn','Dark - No Street Lights','Dark - Street Lights Off','Dark - Unknown Lighting'])
X[:,4] = le_light.transform(X[:,4])

X[0:5]


In [ ]:
y = np.asarray(severity_df['SEVERITYCODE'])
y[0:5]

In [ ]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 4)
print('Train set:', X_train.shape, y_train.shape)
print('Test set:', X_test.shape, y_test.shape)

In [ ]:
LR = LogisticRegression(C=0.01,solver='liblinear').fit(X_train,y_train)
LR

In [ ]:
yhat = LR.predict(X_test)
yhat

In [ ]:
yhat_prob = LR.predict_proba(X_test)
yhat_prob

In [ ]:
from sklearn.metrics import jaccard_similarity_score
jaccard_similarity_score(y_test, yhat)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools
def plot_confusion_matrix(cm, classes, normalize = False, title = 'Confusion matrix', cmap = plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
    print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks=np.arange(len(classes))
    plt.xticks(tick_marks,classes,rotation=45)
    plt.yticks(tick_marks,classes)
    
    fmt='.2f' if normalize else 'd'
    thresh=cm.max() / 2.
    for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
        plt.text(j,i,format(cm[i,j],fmt),
                horizontalalignment="center",
                color="white" if cm[i,j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
print(confusion_matrix(y_test,yhat,labels=[1,0]))

In [ ]:
cnf_matrix = confusion_matrix(y_test,yhat,labels=[1,0])
np.set_printoptions(precision=2)

plt.figure()
plot_confusion_matrix(cnf_matrix,classes=['Injury = 1', 'Injury = 0'],normalize = False, title='Confusion Matrix')

In [ ]:
print (classification_report(y_test,yhat))

In [ ]:
from sklearn.metrics import log_loss
log_loss(y_test,yhat_prob)

In [ ]:
# START SVM Code

from sklearn import svm
clf = svm.SVC(kernel='rbf')
clf.fit(X_train, y_train)

In [ ]:
yhat = clf.predict(X_test)
yhat[0:5]

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import itertools

In [ ]:
def plot_confusion_matrix(cm, classes, normalize = False, title = 'Confusion matrix', cmap = plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')
    
    print(cm)
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks=np.arange(len(classes))
    plt.xticks(tick_marks,classes,rotation=45)
    plt.yticks(tick_marks,classes)
    
    fmt='.2f' if normalize else 'd'
    thresh=cm.max() / 2.
    for i,j in itertools.product(range(cm.shape[0]),range(cm.shape[1])):
        plt.text(j,i,format(cm[i,j],fmt),
                horizontalalignment="center",
                color="white" if cm[i,j] > thresh else "black")
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
print(confusion_matrix(y_test,yhat,labels=[1,0]))

In [ ]:
cnf_matrix = confusion_matrix(y_test, yhat, labels=[1,0])
np.set_printoptions(precision=2)

print(classification_report(y_test,yhat))

plt.figure()
plot_confusion_matrix(cnf_matrix, classes = ['No Injury', 'Injury'], normalize = False, title='Confusion Matrix')
